# Согласованность психотипов по текстовым характеристикам

In [1]:
import sys
sys.path.append("../../")

In [2]:
import warnings
import pingouin as pg
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import json
import numpy as np
import random
import itertools as it
from collections import Counter
from tqdm import tqdm
from analytics_lib.nlp_texts.psychotype import dict_with_psychotype_by_text, dict_norm, dict_with_temp_by_text

/Users/mac-os/Documents/Work/bortnik_psychometry/bortnik/lib/python3.9/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.11, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [29]:
# with open('../data/dict_with_all_texts.json', encoding = "utf8") as f:
#     dict_with_all_texts = json.load(f)
        
# list_of_all_texts = np.concatenate([list(v.values()) for v in dict_with_all_texts.values()])

# list_valid_texts = list_of_all_texts[-16:] # отложенная выборка (не обучаемся на ней)

# list_of_texts = list_of_all_texts[:-16]

In [3]:
public_df = pd.read_pickle("../data/public_df.pkl")
public_semantic_role_df = pd.read_pickle("../data/public_semantic_role_df.pkl")
df_merge = public_df.merge(public_semantic_role_df, left_on='Текст', right_on='Текст', how='inner', suffixes=('_soft','_hard'), indicator = True).sort_values(by=['_merge'], ascending = False, ignore_index = True)
list_of_texts = list(df_merge["Текст"])

In [4]:
len(list_of_texts)

137

In [5]:
df_merge.head(1)

,Текст,Общее количество слов в тексте,Доля предложений на одно слово,Доля коротких предложений,Доля длинных предложений,Доля длинных низкочастотных слов,Доля коротких высокочастотных слов,Средняя длина предложения,Среднее число запятых в предложении,Доля сложноподчиненных предложений,...,Внешний предикат,"Агенс, конструкции","Пациенс, конструкции","Внутренний предикат, конструкции","Внешний предикат, конструкции","Агенс, индексы","Пациенс, индексы","Внутренний предикат, индексы","Внешний предикат, индексы",_merge
0,Сделал бы кривую ценности. Составил бы шаблон ...,39,0.18,0.86,0.14,0.08,0.36,5.57,0.29,0.14,...,0.4,"[[(Сделал,), (Обсудил,), (Предложил,), (Сделал...","[[(Составил,), (составить,), (протестировать,)...",[[]],"[[(Сделал,), (сделал,), (Сделал,)]]","[[(0,), (11,), (36,), (46,)]]","[[(5,), (21,), (27,), (32,), (41,)]]",[[]],"[[(0,), (32,), (46,)]]",both


In [4]:
# text = list_of_texts[0]
# list_all_features = list(df_merge._get_numeric_data().columns)
# list_features = list_all_features[:4]
# df_merge_slice = df_merge[['Текст'] + list_features]
# dict_with_psychotype_by_text(text, df_merge_slice, df_merge_slice, df_merge_slice, df_merge_slice)

In [5]:
def psych_by_text(text, list_features):
    df_merge_slice = df_merge[['Текст'] + list_features]
    dict_n = dict_norm(dict_with_psychotype_by_text(text, df_merge_slice, df_merge_slice, df_merge_slice, df_merge_slice))
    for psych in ["Истероид", "Гипертим", "Шизоид", "Параноял", "Эпилептоид", "Депрессивный", "Эмотив", "Тревожный"]:
        if not dict_n.get(psych):
            dict_n[psych] = 0
    return dict_n

def temp_by_text(text, list_features):
    df_merge_slice = df_merge[['Текст'] + list_features]
    dict_n = dict_norm(dict_with_temp_by_text(text, df_merge_slice, df_merge_slice, df_merge_slice, df_merge_slice))
    for psych in ["Меланхолик", "Холерик", "Флегматик", "Сангвиник"]:
        if not dict_n.get(psych):
            dict_n[psych] = 0
    return dict_n

In [ ]:
list_of_train_texts = list_of_texts
df = pd.DataFrame()
psych = "Истероид"
list_all_features = list(df_merge._get_numeric_data().columns)
size = 5
i = 0
for tuple_features in tqdm(it.combinations(list_all_features, size)):
    if i > 100:
        break
    i += 1
    df[str(list(tuple_features))] = pd.Series(list_of_train_texts).apply(lambda text: psych_by_text(text, list(tuple_features))[psych])

100it [12:25,  7.50s/it]

In [ ]:
df.to_pickle("../data/df_kronbach.pkl")

In [ ]:
score = pg.cronbach_alpha(data=df)
print(score)

In [19]:
%%time
# случайным образом разбиваем выборку на train и test в соотношении 70:30
list_of_train_texts = list_of_texts[:10]
#list_of_test_texts = list_of_texts[252:]

df = pd.DataFrame()
#df['Текст'] = list_of_train_texts
psych = "Истероид"
list_all_features = list(df_merge._get_numeric_data().columns)
list_features1 = list_all_features[:56]
list_features2 = list_all_features[56:]
df[str(list_features1)] = pd.Series(list_of_train_texts).apply(lambda text: psych_by_text(text, list_features1)[psych])
df[str(list_features2)] = pd.Series(list_of_train_texts).apply(lambda text: psych_by_text(text, list_features2)[psych])
df

#     df_v = pd.DataFrame()
#     for token in list_of_test_texts:
#         df_v[token] = pd.Series(list_of_indicators_values(token, df_for_rand_syst_indicators, df_for_rand_syst_indicators))
#     df_v["Индикаторы"] = pd.Series(list_features_names)
#     df_v = df_v.set_index("Индикаторы")
    
#     df_vvv = pd.DataFrame()
#     for token in list_valid_texts:
#         df_vvv[token] = pd.Series(list_of_indicators_values(token, df_for_rand_syst_indicators, df_for_rand_syst_indicators))
#     df_vvv["Индикаторы"] = pd.Series(list_features_names)
#     df_vvv = df_vvv.set_index("Индикаторы")
    
#     for size in range(2, 8):
#         for x in it.combinations(list_features_names, size):
#             score = pg.cronbach_alpha(data=df.loc[list(x)])
#             if score[0] >= 0.7:
#                 score_test = pg.cronbach_alpha(data=df_v.loc[list(x)])
#                 if score_test[0] >= 0.7:
#                     dict_res = {}
#                     dict_res["№ Разбиения"] = i
#                     dict_res["alpha-train"] = score[0] # согласованность на тренировочной выборке
#                     dict_res["alpha-test"] = score_test[0] # согласованность на тестовой выборке
#                     score_vvv = pg.cronbach_alpha(data=df_vvv.loc[list(x)])
#                     dict_res["alpha-valid"] = score_vvv[0] # проверка согласованность на отложенной выборке
#                     dict_res["Индикаторы"] = list(x)
#                     dict_for_df = {k[0]: [k[1]] for k in dict_res.items()}
#                     df_temp = pd.DataFrame.from_dict(dict_for_df)
#                     df_res = df_res.append(df_temp, ignore_index=True)

CPU times: user 5.62 s, sys: 7.94 ms, total: 5.63 s
Wall time: 5.63 s


,"['Общее количество слов в тексте', 'Доля предложений на одно слово', 'Доля коротких предложений', 'Доля длинных предложений', 'Доля длинных низкочастотных слов', 'Доля коротких высокочастотных слов', 'Средняя длина предложения', 'Среднее число запятых в предложении', 'Доля сложноподчиненных предложений', 'Доля восклицательных предложений', 'Доля вопросительных предложений', 'Доля частиц НЕ', 'Cредняя длинна слова в тексте', 'Гласные, %', 'Гласная ""И"", %', 'Гласная ""О"", %', 'Гласная ""А"", %', 'Согласные, %', 'Существительные, %', 'Глаголы, %', 'Совершенный вид, %', 'Несовершенный вид, %', 'Прилагательные, %', 'Степени сравнения, %', 'Превосходная степень, %', 'Сравнительная степень, %', 'Наречия, %', 'Причастия, %', 'Деепричастия, %', 'Наклонения, %', 'Изъявительное наклонение, %', 'Условное наклонение, %', 'Повелительное наклонение, %', 'Среднее количество слов в предложении', 'Средняя доля именованных сущностей в предложении', 'Среднее количество морфем в слове', 'Средняя частота TOP10 самых редких слов в тексте', 'Среднее число синтаксических веток (по предложениям)', 'Среднее число синтаксических веток (на одно слово)', 'Средняя глубина синтаксической ветки', 'Среднее число прилагательных в синтаксической ветке', 'Среднее число причастий в синтаксической ветке', 'Среднее число деепричастий в синтаксической ветке', 'Водность текста', 'Вариабельность словаря', 'Доля повторов в тексте', 'Доля местоимений Я', 'Классическая тошнота', 'Академическая тошнота', 'Доля местоимений в тексте', 'Доля частиц в тексте', 'Собственные/Нарицательные', 'Доля числительных в тексте', 'Доля междометий в тексте', 'Доля союзов в тексте', 'Доля предлогов в тексте']","['Агенс', 'Пациенс', 'Внутренний предикат', 'Внешний предикат']"
0,0.17,0.17
1,0.26,0.20
2,0.06,0.25
3,0.30,0.00
4,0.11,0.12
5,0.23,0.25
6,0.17,0.20
7,0.24,0.20
8,0.30,0.00
9,0.21,0.00


In [28]:
score = pg.cronbach_alpha(data=df)
print(score)

(1.0, array([1., 1.]))


In [9]:
df

,№ Разбиения,alpha-train,alpha-test,alpha-valid,Индикаторы
0,1,0.858667,0.779727,-inf,"[Несовершенный вид / Совершенный вид, Доля час..."
1,1,0.849492,0.852545,-1.706667e+01,"[Количество наречий / Количество деепричастий,..."
2,1,0.724074,0.800000,8.533333e-02,[Количество причастий / Количество деепричасти...
3,1,0.812948,0.746040,-3.306667e+01,"[Несовершенный вид / Совершенный вид, Количест..."
4,1,0.738067,0.730204,2.368476e-16,"[Количество наречий / Количество деепричастий,..."
5,1,0.757685,0.716224,-1.148718e+00,"[Количество наречий / Количество деепричастий,..."
6,2,0.806597,0.715789,0.000000e+00,"[Доля гласных / Доля согласных, Несовершенный ..."
7,2,0.874000,0.701754,-inf,"[Несовершенный вид / Совершенный вид, Доля час..."
8,2,0.813533,0.896917,-1.706667e+01,"[Количество наречий / Количество деепричастий,..."
9,2,0.846528,0.756579,-4.740741e-01,"[Доля частиц НЕ среди всех слов, Мелодика]"


In [10]:
df_res.to_csv("../data/alpha_33.csv", index = False) # сохраняем в csv

# Формируем соответствующую таблицу с обратными квантилями и значениями индикаторов

In [11]:
def feature_value_by_token(token, feature_name, person_df):
    feature_value = person_df[person_df["token"] == token][feature_name].values[0]
    return feature_value

In [12]:
def inverse_percentile(arr, num):
    arr = sorted(arr)
    i_arr = [i for i, x in enumerate(arr) if x > num]
    return i_arr[0] / len(arr) if len(i_arr) > 0 else 1

In [13]:
def table_by_token(token, person_df, df_for_rand_syst_indicators):
    list_of_inverse_quantile = []
    list_of_feature_name = []
    list_of_ind_value = []
    dict_temp = {}
    
    feature_name = 'Доля гласных / Доля согласных'
    list_of_feature_name.append(feature_name)
    feature_value = feature_value_by_token(token, feature_name, person_df)
    list_of_inverse_quantile.append(inverse_percentile(df_for_rand_syst_indicators[feature_name].to_list(), feature_value))
    if indicator_large(token, feature_name, person_df, df_for_rand_syst_indicators):
        list_of_ind_value.append(0)
    elif indicator_mean(token, feature_name, person_df, df_for_rand_syst_indicators):
        list_of_ind_value.append(1)
    else:
        list_of_ind_value.append(2)
        
    feature_name = 'Несовершенный вид / Совершенный вид'
    list_of_feature_name.append(feature_name)
    feature_value = feature_value_by_token(token, feature_name, person_df)
    list_of_inverse_quantile.append(inverse_percentile(df_for_rand_syst_indicators[feature_name].to_list(), feature_value))
    if indicator_large(token, feature_name, person_df, df_for_rand_syst_indicators):
        list_of_ind_value.append(0)
    elif indicator_mean(token, feature_name, person_df, df_for_rand_syst_indicators):
        list_of_ind_value.append(1)
    else:
        list_of_ind_value.append(2)
        
    feature_name = 'Количество глаголов / Количество прилагательных'
    list_of_feature_name.append(feature_name)
    feature_value = feature_value_by_token(token, feature_name, person_df)
    list_of_inverse_quantile.append(inverse_percentile(df_for_rand_syst_indicators[feature_name].to_list(), feature_value))
    if indicator_little(token, feature_name, person_df, df_for_rand_syst_indicators):
        list_of_ind_value.append(0)
    elif indicator_mean(token, feature_name, person_df, df_for_rand_syst_indicators):
        list_of_ind_value.append(1)
    else:
        list_of_ind_value.append(2)  
        
    feature_name = 'Количество наречий / Количество деепричастий'
    list_of_feature_name.append(feature_name)
    feature_value = feature_value_by_token(token, feature_name, person_df)
    list_of_inverse_quantile.append(inverse_percentile(df_for_rand_syst_indicators[feature_name].to_list(), feature_value))
    if indicator_large(token, feature_name, person_df, df_for_rand_syst_indicators):
        list_of_ind_value.append(0)
    elif indicator_mean(token, feature_name, person_df, df_for_rand_syst_indicators):
        list_of_ind_value.append(1)
    else:
        list_of_ind_value.append(2)
        
    feature_name = 'Количество причастий / Количество деепричастий'
    list_of_feature_name.append(feature_name)
    feature_value = feature_value_by_token(token, feature_name, person_df)
    list_of_inverse_quantile.append(inverse_percentile(df_for_rand_syst_indicators[feature_name].to_list(), feature_value))
    if indicator_large(token, feature_name, person_df, df_for_rand_syst_indicators):
        list_of_ind_value.append(0)
    elif indicator_mean(token, feature_name, person_df, df_for_rand_syst_indicators):
        list_of_ind_value.append(1)
    else:
        list_of_ind_value.append(2)
        
    feature_name = 'Доля условного наклонения / Доля повелительного наклонения'
    list_of_feature_name.append(feature_name)
    feature_value = feature_value_by_token(token, feature_name, person_df)
    list_of_inverse_quantile.append(inverse_percentile(df_for_rand_syst_indicators[feature_name].to_list(), feature_value))
    if indicator_large(token, feature_name, person_df, df_for_rand_syst_indicators):
        list_of_ind_value.append(0)
    elif indicator_mean(token, feature_name, person_df, df_for_rand_syst_indicators):
        list_of_ind_value.append(1)
    else:
        list_of_ind_value.append(2)
        
    feature_name = 'Доля частиц НЕ среди всех слов'
    list_of_feature_name.append(feature_name)
    feature_value = feature_value_by_token(token, feature_name, person_df)
    list_of_inverse_quantile.append(inverse_percentile(df_for_rand_syst_indicators[feature_name].to_list(), feature_value))
    if indicator_large(token, feature_name, person_df, df_for_rand_syst_indicators):
        list_of_ind_value.append(2)
    else:
        list_of_ind_value.append(1)
        
    feature_name = 'Мелодика'
    melodic = 0
    list_of_feature_name.append(feature_name)
    feature_value = feature_value_by_token(token, feature_name, person_df)
    list_of_inverse_quantile.append(inverse_percentile(df_for_rand_syst_indicators[feature_name].to_list(), feature_value))
    if indicator_little(token, feature_name, person_df, df_for_rand_syst_indicators):
        list_of_ind_value.append(0)
        melodic = 0
    elif indicator_mean(token, feature_name, person_df, df_for_rand_syst_indicators):
        list_of_ind_value.append(1)
        melodic = 1
    else:
        list_of_ind_value.append(2)
        melodic = 2
        
    feature_name = 'Хаотичная/Системная'
    list_of_feature_name.append(feature_name)
    counter = Counter(list_of_ind_value)
    if counter[2] > counter[0]:
        feature_value = 2
    elif counter[0] > counter[2]:
        feature_value = 0
    else:
        feature_value = 1
    list_of_inverse_quantile.append(-1)
    list_of_ind_value.append(feature_value)
    
    feature_name = 'Хаотичная/Системная (Мелодика*2)'
    list_of_feature_name.append(feature_name)
    list_of_ind_value_new = list_of_ind_value[:-1] + [melodic]
    counter1 = Counter(list_of_ind_value_new)
    if counter1[2] > counter1[0]:
        feature_value = 2
    elif counter1[0] > counter1[2]:
        feature_value = 0
    else:
        feature_value = 1
    list_of_inverse_quantile.append(-1)
    list_of_ind_value.append(feature_value)
    
    
    dict_temp = {"Индикатор": list_of_feature_name, "Значение": list_of_ind_value, "Обратный квантиль": np.round(list_of_inverse_quantile, 2)}
    df = pd.DataFrame(dict_temp)
    
    return df

# Вывод таблиц со значениями индикаторов шкалы "Хаотичная/Системная" и обратными квантилями по всем имеющимся в БД тестам

In [15]:
for token in list_of_all_tokens:
    print("UserId:", token)
    print("\n" + df_for_rand_syst_indicators[df_for_rand_syst_indicators['token'] == token]["Текст"].values[0])
    display(table_by_token(token, df_for_rand_syst_indicators, df_for_rand_syst_indicators))
    print("\n\n\n")

UserId: pptAD3zMAjgChkWUPGmw7V

Провести презентацию, рассказать о преимуществах и новых возможностях данного продукта, дать возможность попробовать новинку в действии и получить обратную связь для дальнейших переговоров, в случае положительной обратной связи выйти на сделку для ввода данного продукта в ассортимент клиента, в случае отрицательной обратной связи предложить попробовать альтернативу

Провести переговоры с консервативными клиентами на языке выгод и договориться о вводе продукта для наглядности работы, что бы клиент понимал для чего данный продукт создан и почему ему выгодно пользоваться именно им, найти компромисс в условиях поставки для комфортного сотрудничества по вводу новинки

Анализировать где именно развивается конкурент, почему он развивается именно у этих клиентов, что предлагает клиентам для комфортного сотрудничества, затем необходимо собрать обратную связь от потенциальных клиентов о + и - сотрудничества с конкурентом, предложить сервис и более комфортные услов

,Индикатор,Значение,Обратный квантиль
0,Доля гласных / Доля согласных,2,0.01
1,Несовершенный вид / Совершенный вид,1,0.44
2,Количество глаголов / Количество прилагательных,0,0.27
3,Количество наречий / Количество деепричастий,1,0.35
4,Количество причастий / Количество деепричастий,1,0.48
5,Доля условного наклонения / Доля повелительног...,0,0.96
6,Доля частиц НЕ среди всех слов,1,0.18
7,Мелодика,2,0.79
8,Хаотичная/Системная,1,-1.00
9,Хаотичная/Системная (Мелодика*2),2,-1.00






UserId: 73JPFxR7eRT764TxkWDdg7

Необходимо рассказать о продукте большому количеству людей, друзьям знакомым, поделится информацией в соц. сетях, возможно люди сразу не пойдут на то, чтобы выслушать и узнать, что-то новое, но необходимо презентовать всем и при первой возможности, даже если это будет просто информация, это скорее всего поможет продавать продукт в будущем.

Необходимо подходить с точки зрения прогресса во всём, все нуждаются в комфортных продуктах, необходимо презентовать выгоды продукта, экономически обосновывать покупку продукта, в правильном ключе вести диалог о том, что не нужно упускать возможность приобрести на старте, это поможет сэкономить время и деньги.

Во первых необходимо поддерживать активную клиентскую базу, поддерживать личный контакт с клиентами, чтобы у клиента было ощущение комфорта в работе с нами, ощущение о том, что о нем не забыли, рассказывать о новинках продукта. Изучать продукт конкурентов, все его плюсы и минусы, чтобы вести конструктивный 

,Индикатор,Значение,Обратный квантиль
0,Доля гласных / Доля согласных,2,0.10
1,Несовершенный вид / Совершенный вид,0,0.80
2,Количество глаголов / Количество прилагательных,2,0.79
3,Количество наречий / Количество деепричастий,0,0.98
4,Количество причастий / Количество деепричастий,1,0.54
5,Доля условного наклонения / Доля повелительног...,2,0.52
6,Доля частиц НЕ среди всех слов,2,0.91
7,Мелодика,2,0.99
8,Хаотичная/Системная,2,-1.00
9,Хаотичная/Системная (Мелодика*2),2,-1.00






UserId: ud2g87FD4GtKU3Pt6xMYve

Выясняем уникальное торговое предложение данного инновационного продукта. Анализируем у каких клиентов возможно использование данного продукта. Выявляем их возможные потребности. Делаем презентацию и отдаем им на испытания. Далее на основе испытаний и УТП продаем выгоду от нашего продукта клиенту и сам продукт.

Выявляем потребности у консервативных клиентов. Проводим им презентацию и демонстрацию продукта. Так же используя примеры третьих лиц(положительные результаты других испытаний) и используя УТП продукта отдаем продукт на испытания. Получаем обратную связь. В случае положительной обратной связи производим продажу продукта.

Проверить сделки на каком этапе находиться текущие сделки. Возможно некоторые сделки были забыты и требуют вмешательства. Проанализировать новые каналы привлечения клиента. Проработать ассортиментную матрицу каждого клиента. Проработать их. Так же  необходимо провести анализ возможных продаж конкурента. Выявить точки роста к

...

,Индикатор,Значение,Обратный квантиль
0,Доля гласных / Доля согласных,2,0.21
1,Несовершенный вид / Совершенный вид,2,0.09
2,Количество глаголов / Количество прилагательных,0,0.30
3,Количество наречий / Количество деепричастий,1,0.43
4,Количество причастий / Количество деепричастий,0,0.83
5,Доля условного наклонения / Доля повелительног...,2,0.52
6,Доля частиц НЕ среди всех слов,1,0.43
7,Мелодика,2,0.98
8,Хаотичная/Системная,2,-1.00
9,Хаотичная/Системная (Мелодика*2),2,-1.00






UserId: garazh_km_130421-4

Идея возникает не на пустом месте. Значит накопился достаточно глубокий объем знаний по продукту, когда основные и базовые потребности закрыты, но возникают неудобства связанные с конкретными пользователями, с их индивидуальными операциями, потребностями в решении регулярных задач. Идею нужно проверять на соответствующем потребителе, которому эта идея должна быть полезна. Описывать идеи нужно ярко, лучше в сравнении с тем как было и как может стать (отличный пример, - реклама таблеток для похудения, слева модель плюс сайза, справа модель скинни)

"Тогда я устрою свое казино, со своим блекджеком и со своими куртизанками". На самом деле важно понять суть сопротивления коллег и руководства, чем оно продиктовано. Чаще всего проблема лежит в плоскости необходимости принятия решений, а их принимать не хочется (старая инженерная поговорка , - "если это работает - не трогай") или это приводит к конфликту интересов, например затрагивается чужая мотивация, ну или 

,Индикатор,Значение,Обратный квантиль
0,Доля гласных / Доля согласных,1,0.68
1,Несовершенный вид / Совершенный вид,0,0.85
2,Количество глаголов / Количество прилагательных,1,0.37
3,Количество наречий / Количество деепричастий,0,0.71
4,Количество причастий / Количество деепричастий,0,0.95
5,Доля условного наклонения / Доля повелительног...,1,0.60
6,Доля частиц НЕ среди всех слов,2,0.99
7,Мелодика,2,0.82
8,Хаотичная/Системная,0,-1.00
9,Хаотичная/Системная (Мелодика*2),1,-1.00






UserId: garazh_km_130421-11

Как бы я описывал идею …? Каким образом …? Такой вопрос …?
Используя иллюстрации, таблицы, схемы максимально доступно отражающие целевые области и сферы применения продукта.
Про протестировать: Предположил бы теорию для начала, так как теория позволяет задавать правильные вопросы. Провел бы опрос среди максимально приближенных и хорошо разбирающихся в теме исследования, затем провел бы реальные испытания. Таким образом вне зависимости от исхода испытания или теста, получил бы знания, опираясь на которые можно корректировать теорию или предполагать новую, становясь всё ближе цели.

Буду выяснять какие есть основания для сопротивления и у кого. Сопоставляя мои "за" и общие "против" можно отыскать компромисс скорректировав конечный продукт и/или процессы которые он затронет и в конечном итоге двигаться дальше в сторону внедрения, которое неизбежно предполагая что "эксперимент показал положительные результаты".

Действия по управлению продуктом так или инач

,Индикатор,Значение,Обратный квантиль
0,Доля гласных / Доля согласных,0,0.95
1,Несовершенный вид / Совершенный вид,0,0.88
2,Количество глаголов / Количество прилагательных,1,0.61
3,Количество наречий / Количество деепричастий,2,0.11
4,Количество причастий / Количество деепричастий,2,0.16
5,Доля условного наклонения / Доля повелительног...,0,0.95
6,Доля частиц НЕ среди всех слов,1,0.51
7,Мелодика,0,0.24
8,Хаотичная/Системная,0,-1.00
9,Хаотичная/Системная (Мелодика*2),0,-1.00
